<a href="https://colab.research.google.com/github/Alok130802/Image-Classification-using-CNN---Fruit360/blob/master/fruit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import data from google drive

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade tensorflow==2.0

     |████████████████████████████████| 86.3MB 53kB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
     |████████████████████████████████| 450kB 69.9MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

## Unzip fruit dataset in folder fruitdata

In this cell first we will unzip our zipped fruit dataset 

In [4]:
from pathlib import Path
homedir = Path("./drive/My Drive/fruits.zip")



from zipfile import ZipFile
with ZipFile("./drive/My Drive/fruits.zip",'r') as zip:
    print("started unziping ....")
    zip.extractall()
    print("ended unziping ...")



started unziping ....
ended unziping ...


In [0]:
trainDir = Path('./fruits-360_dataset/fruits-360/Training')
testDir = Path('./fruits-360_dataset/fruits-360/Test')
listOfAllFruitsFolder = list()
for i in trainDir.iterdir():
    listOfAllFruitsFolder.append(i)


In [0]:
import tensorflow as tf
from tensorflow.keras import layers
tf.__version__
from PIL import Image
import matplotlib.pyplot as plt

In [0]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

## create a train image generator

In [8]:
 
train_genertaor =tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./225)

train_Dataset = train_genertaor.flow_from_directory(trainDir,target_size=(50,50),shuffle = True)
sample_image,label = next(train_Dataset)
img = label[0]
print(img.shape)

Found 60498 images belonging to 120 classes.
(120,)


## create a model given in paper 


In [9]:
## create a model given in paper 

model  = tf.keras.models.Sequential()

model.add(layers.Conv2D(16,(5,5),padding='same',activation='relu',input_shape=(50,50,3)))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.Conv2D(32,(5,5),padding='same',activation='relu'))

model.add(layers.MaxPool2D(strides=2))
model.add(layers.Conv2D(64,(5,5),padding='same',activation='relu'))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.Conv2D(128,(5,5),padding='same',activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(120,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 16)        1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 128)         2

## Compile Model using adam as optimizer 

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [12]:
epoch = 10
history = model.fit_generator(train_Dataset,epochs=epoch )

Epoch 1/10
1891/1891 [==============================] - 543s 287ms/step - loss: 0.1428 - categorical_accuracy: 0.9564
Epoch 2/10
1891/1891 [==============================] - 529s 280ms/step - loss: 0.0678 - categorical_accuracy: 0.9821
Epoch 3/10
1891/1891 [==============================] - 515s 272ms/step - loss: 0.0416 - categorical_accuracy: 0.9885
Epoch 4/10
1891/1891 [==============================] - 496s 262ms/step - loss: 0.0485 - categorical_accuracy: 0.9884
Epoch 5/10
1891/1891 [==============================] - 496s 262ms/step - loss: 0.0321 - categorical_accuracy: 0.9919
Epoch 6/10
1891/1891 [==============================] - 502s 265ms/step - loss: 0.0394 - categorical_accuracy: 0.9903
Epoch 7/10
1891/1891 [==============================] - 511s 270ms/step - loss: 0.0338 - categorical_accuracy: 0.9923
Epoch 8/10
1891/1891 [==============================] - 488s 258ms/step - loss: 0.0409 - categorical_accuracy: 0.9922
Epoch 9/10
1891/1891 [==============================] - 

## Save Model on disk 

In [0]:
model.save('fruit_model.h5')
new_model = tf.keras.models.load_model('fruit_model.h5')


## Evaluate Model 

In [16]:
test_genertaor =tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./225)

test_Dataset = test_genertaor.flow_from_directory(testDir,target_size=(50,50),shuffle = True)
loss,acc = new_model.evaluate_generator(test_Dataset,verbose=2)


Found 20622 images belonging to 120 classes.
645/645 - 46s - loss: 0.4961 - categorical_accuracy: 0.9368


[0.4960996330451897, 0.936815]

## At last we got the 93 % Test Accuracy